In [ ]:
import glob

listOfFileNames = []
listOfClassNames = []
listOfDataPerClass = []
files = []
classNames = []

classNames = glob.glob("/Users/user/Desktop/datathesis/placeholder/*")

for c in classNames:
    className = c.split("\\")[1]
    listOfClassNames.append(c.split("\\")[1])
    files = glob.glob("/Users/user/Desktop/Thesis/placeholder/"+className+"/*.csv")
    listOfFileNames = []
    for f in files:
        listOfFileNames.append(f.split("\\")[1])
    listOfDataPerClass.append(listOfFileNames)

    
#listOfClassNames
#listOfDataPerClass[0]
print("Done listing the class and file names")

In [ ]:
notfirstday = []
for i in listOfClassNames:
    if i not in firstDay:
        notfirstday.append(i)

notsday = []
for i in notfirstday:
    if i not in secondDay1:
        notsday.append(i)
        
notsday2 = []
for i in notsday:
    if i not in secondDay2:
        notsday2.append(i)

nottday = []
for i in notsday2:
    if i not in thirdDay:
        nottday.append(i)

print(nottday)
print(len(listOfClassNames))

print(len(firstDay))
print(len(secondDay1))
print(len(secondDay2))
print(len(thirdDay))
print(len(firstDay)+len(secondDay1)+len(secondDay2)+len(thirdDay))

In [ ]:
#len(listOfDataPerClass)
#listOfDataPerClass[3]

import pandas as pd


listOfDataFramesPerClass = {}
dataFrameDictionary = {}
dataFrames = []

for c, d in zip(listOfClassNames, listOfDataPerClass):
    print("----------------- Word " + c + "-----------------")
    emgDataFrames = {}
    emgDataFrames['right'] = []
    emgDataFrames['left'] = []
    
    accDataFrames = {}
    accDataFrames['right'] = []
    accDataFrames['left'] = []

    gyroDataFrames = {}
    gyroDataFrames['right'] = []
    gyroDataFrames['left'] = []

    dataFrameDictionary = {}
    className = c
    leftctr = 0
    rightctr = 0
    for fileName in d:
        fileLocation = "/Users/user/Desktop/Thesis/placeholder/"+className+"/"+fileName + ""
        df = pd.read_csv(fileLocation)
        
        if "emg" in fileName:   
            if "-0-" in fileName:
                emgDataFrames['right'].append(df)
                leftctr+=1
            else:
                emgDataFrames['left'].append(df)
                rightctr+=1
        elif ("accelerometer") in fileName:
            if "-0-" in fileName:
                accDataFrames['right'].append(df)
                leftctr+=1
            else:
                accDataFrames['left'].append(df)
                rightctr+=1
        elif ("gyro") in fileName:
            if "-0-" in fileName:
                gyroDataFrames['right'].append(df)
                leftctr+=1
            else:
                gyroDataFrames['left'].append(df)
                rightctr+=1
    
    dataFrameDictionary["emg"] = emgDataFrames
    dataFrameDictionary["acc"] = accDataFrames
    dataFrameDictionary["gyro"] = gyroDataFrames

    listOfDataFramesPerClass[className] = dataFrameDictionary
    print("Total of " + str(leftctr) + " left arm files and " + str(rightctr) + " right arm files.")


print("Done converting to dataframes!")


## For EMG Values

In [ ]:
for a in listOfDataFramesPerClass[k]['emg']:
    print(a)

# Trimming

In [ ]:
import pandas as pd

listOfTrimmedDataFramesPerArm = {}
listOfTrimmedDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['emg']:
        for d in listOfDataFramesPerClass[k]['emg'][a]:
            keep_col = ['emg1','emg2','emg3','emg4','emg5','emg6','emg7','emg8']
            new_df = d[keep_col]
            new_df = new_df[:-49]
            trimmed_df = new_df.iloc[49:]
            if len(trimmed_df) < 100:
                print(str(ctr) + " " + k)
            trimmedDataFramesPerClass[a].append(trimmed_df)
            ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["emg"] = trimmedDataFramesPerClass
    listOfTrimmedDataFramesPerClass[k] = dataFrameDictionary

# Normalization

In [ ]:
from sklearn import preprocessing

listOfNormalizedDataFramesPerClass = {}

for k in listOfTrimmedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = {}
    NormalizedDataFramesPerClass['right'] = []
    NormalizedDataFramesPerClass['left'] = []
    
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['emg']:
        ctr = 0
        for d in listOfDataFramesPerClass[k]['emg'][a]:
            #print(str(len(d.index)) + " " + str(ctr))
            # Create x, where x the 'scores' column's values as floats
            x = d.values.astype(float)

            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()

            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)

            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            #df_normalized
            NormalizedDataFramesPerClass[a].append(df_normalized)
            ctr += 1
        
        print("Done Normalizing " + str(ctr) + a + " arm files")
    dataFrameDictionary["emg"] = NormalizedDataFramesPerClass
    listOfNormalizedDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



In [ ]:
import pandas as pd
from statsmodels import robust
import numpy as np

listOfVarianceDataFramePerClass = {}
listOfVarianceDataFramePerClass['right'] = []
listOfVarianceDataFramePerClass['left'] = []

listOfMavDataFramePerClass = {}
listOfMavDataFramePerClass['right'] = []
listOfMavDataFramePerClass['left'] = []

listOfMeanDataFramePerClass = {}
listOfMeanDataFramePerClass['right'] = []
listOfMeanDataFramePerClass['left'] = []

listOfSdDataFramePerClass = {}
listOfSdDataFramePerClass['right'] = []
listOfSdDataFramePerClass['left'] = []

listOfRmsDataFramePerClass = {}
listOfRmsDataFramePerClass['right'] = []
listOfRmsDataFramePerClass['left'] = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    for a in listOfNormalizedDataFramesPerClass[k]['emg']:
        ctr = 0
        varianceDataFrame = pd.DataFrame(columns = ['Variance of EMG1', 'Variance of EMG2', 'Variance of EMG3', 'Variance of EMG4',
                                 'Variance of EMG5', 'Variance of EMG6', 'Variance of EMG7', 'Variance of EMG8'])
        mavDataFrame = pd.DataFrame(columns = ['Mav of EMG1', 'Mav of EMG2', 'Mav of EMG3', 'Mav of EMG4',
                                     'Mav of EMG5', 'Mav of EMG6', 'Mav of EMG7', 'Mav of EMG8'])
        meanDataFrame = pd.DataFrame(columns = ['Mean of EMG1', 'Mean of EMG2', 'Mean of EMG3', 'Mean of EMG4',
                                     'Mean of EMG5', 'Mean of EMG6', 'Mean of EMG7', 'Mean of EMG8'])
        sdDataFrame = pd.DataFrame(columns = ['Sd of EMG1', 'Sd of EMG2', 'Sd of EMG3', 'Sd of EMG4',
                                     'Sd of EMG5', 'Sd of EMG6', 'Sd of EMG7', 'Sd of EMG8'])
        rmsDataFrame = pd.DataFrame(columns = ['Rms of EMG1', 'Rms of EMG2', 'Rms of EMG3', 'Rms of EMG4',
                                     'Rms of EMG5', 'Rms of EMG6', 'Rms of EMG7', 'Rms of EMG8'])
        for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg'][a]:
            varianceDataFrame.loc[ctr] = [df_normalized.var()[0], df_normalized.var()[1], df_normalized.var()[2], df_normalized.var()[3], df_normalized.var()[4], df_normalized.var()[5], df_normalized.var()[6], df_normalized.var()[7]]
            mavDataFrame.loc[ctr] = [robust.mad(df_normalized)[0], robust.mad(df_normalized)[1], robust.mad(df_normalized)[2], robust.mad(df_normalized)[3], robust.mad(df_normalized)[4], robust.mad(df_normalized)[5], robust.mad(df_normalized)[6], robust.mad(df_normalized)[7]]
            meanDataFrame.loc[ctr] = [np.mean(df_normalized)[0], np.mean(df_normalized)[1], np.mean(df_normalized)[2], np.mean(df_normalized)[3], np.mean(df_normalized)[4], np.mean(df_normalized)[5], np.mean(df_normalized)[6], np.mean(df_normalized)[7]]
            sdDataFrame.loc[ctr] = [np.std(df_normalized)[0], np.std(df_normalized)[1], np.std(df_normalized)[2], np.std(df_normalized)[3], np.std(df_normalized)[4], np.std(df_normalized)[5], np.std(df_normalized)[6], np.std(df_normalized)[7]]
            rmsDataFrame.loc[ctr] = [np.sqrt(np.mean(df_normalized[0].values**2)), np.sqrt(np.mean(df_normalized[1].values**2)), np.sqrt(np.mean(df_normalized[2].values**2)), np.sqrt(np.mean(df_normalized[3].values**2)), np.sqrt(np.mean(df_normalized[4].values**2)), np.sqrt(np.mean(df_normalized[5].values**2)), robust.mad(df_normalized)[6], np.sqrt(np.mean(df_normalized[7].values**2))]
            ctr +=1 
            
        print(len(varianceDataFrame))
        listOfVarianceDataFramePerClass[a].append(varianceDataFrame)
        listOfMavDataFramePerClass[a].append(mavDataFrame)
        listOfMeanDataFramePerClass[a].append(meanDataFrame)
        listOfSdDataFramePerClass[a].append(sdDataFrame)
        listOfRmsDataFramePerClass[a].append(rmsDataFrame)
    
        print("Done computing the var, mav, mean , sd, and rms of " + str(ctr) + a + " arm files")


In [ ]:
import pandas as pd
import numpy as np

listOfMeanFreqDataFramePerClass = {}
listOfMeanFreqDataFramePerClass['right'] = []
listOfMeanFreqDataFramePerClass['left'] = []

listOfMedFreqDataFramePerClass = {}
listOfMedFreqDataFramePerClass['right'] = []
listOfMedFreqDataFramePerClass['left'] = []

for k in listOfNormalizedDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    
    
    for a in listOfNormalizedDataFramesPerClass[k]['emg']:
        ctr = 0
        meanFreqDataFrame = pd.DataFrame(columns = ['MeanFreq of EMG1', 'MeanFreq of EMG2', 'MeanFreq of EMG3', 'MeanFreq of EMG4',
                                 'MeanFreq of EMG5', 'MeanFreq of EMG6', 'MeanFreq of EMG7', 'MeanFreq of EMG8']) 

        medFreqDataFrame = pd.DataFrame(columns = ['MedianFreq of EMG1', 'MedianFreq of EMG2', 'MedianFreq of EMG3', 'MedianFreq of EMG4',
                                     'MedianFreq of EMG5', 'MedianFreq of EMG6', 'MedianFreq of EMG7', 'MedianFreq of EMG8'])

        for df_normalized in listOfNormalizedDataFramesPerClass[k]['emg'][a]:
            meanFreq = []
            medFreq = []

            for i in range(0, 8):
                data = df_normalized[i].values
                ps = np.abs(np.fft.fft(data))**2

                freqs = np.fft.fftfreq(data.size, 1/100)

                sumProd = 0
                sumPs = 0

                for f, p in zip(freqs,ps):
                    sumProd += f*p
                    sumPs += p

                meanFreq.append(sumProd/sumPs)
                medFreq.append(sumPs/2)

            meanFreqDataFrame.loc[ctr] = [meanFreq[0], meanFreq[1], meanFreq[2], meanFreq[3], meanFreq[4], meanFreq[5], meanFreq[6], meanFreq[7]]
            medFreqDataFrame.loc[ctr] = [medFreq[0], medFreq[1], medFreq[2], medFreq[3], medFreq[4], medFreq[5], medFreq[6], medFreq[7]]

            ctr+=1
        listOfMeanFreqDataFramePerClass[a].append(meanFreqDataFrame)
        listOfMedFreqDataFramePerClass[a].append(medFreqDataFrame)
        print("Done computing Mean and Median Frequency of " + str(ctr) + a + " arm files")
            

Label Class

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])
ctr = 0

listOfLabeledDataPerClass = {}
listOfLabeledDataPerClass['right'] = {}
listOfLabeledDataPerClass['left'] = {}


for a in listOfNormalizedDataFramesPerClass[k]['emg']:

    dataFramesPerFeature = {}
    ctr = 0
    for k in listOfTrimmedDataFramesPerClass.keys():
        dataFramesPerFeature['variance'] = listOfVarianceDataFramePerClass[a][ctr]
        dataFramesPerFeature['mav'] = listOfMavDataFramePerClass[a][ctr]
        dataFramesPerFeature['rms'] = listOfRmsDataFramePerClass[a][ctr]
        dataFramesPerFeature['sd'] = listOfSdDataFramePerClass[a][ctr]
        dataFramesPerFeature['mean'] = listOfMeanDataFramePerClass[a][ctr]
        dataFramesPerFeature['medFreq'] = listOfMedFreqDataFramePerClass[a][ctr]
        dataFramesPerFeature['meanFreq'] = listOfMeanFreqDataFramePerClass[a][ctr]
    

        for k2 in dataFramesPerFeature.keys():
            if k2 != 'variance':
                for i in dataFramesPerFeature[k2].columns:
                    dataFramesPerFeature['variance'][i] = dataFramesPerFeature[k2][i]
        
        dataFramesPerFeature['variance']['Class'] = k
        listOfLabeledDataPerClass[a][k] = dataFramesPerFeature['variance']
        print("Done labeling emg data of " + k +" "+ a + " arm files")
        ctr += 1
 

Data Consolidation (save to csv)

In [ ]:

for a in listOfNormalizedDataFramesPerClass[k]['emg']:
    newDf = listOfLabeledDataPerClass[a]['ano']
    for k in listOfTrimmedDataFramesPerClass.keys():
        if k != 'ano':
            print(k)
            newDf= newDf.append(listOfLabeledDataPerClass[a][k])
    newDf.to_csv('/Users/user/Desktop/'+ a + 'ArmConsolidatedEMG.csv', encoding='utf-8', index=False)

## For IMU values

Trimming (For Acceleration and Gyroscope)

In [ ]:
listOfDataFramesPerClass['isa']['gyro']['right']

In [ ]:
import pandas as pd

listOfTrimmedAccDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}
    
    for a in listOfDataFramesPerClass[k]['emg']:
        for d in listOfDataFramesPerClass[k]['acc'][a]:
            keep_col = ['x','y','z']
            new_df = d[keep_col]
            new_df = new_df[:-24]
            trimmed_df = new_df.iloc[24:]
            if len(trimmed_df) < 50:
                print(str(ctr) + " " + str(len(trimmed_df)))
            trimmedDataFramesPerClass[a].append(trimmed_df)
            ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["acc"] = trimmedDataFramesPerClass
    listOfTrimmedAccDataFramesPerClass[k] = dataFrameDictionary

In [ ]:
import pandas as pd

listOfTrimmedGyroDataFramesPerClass = {}

for k in listOfDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}
    
    for a in listOfDataFramesPerClass[k]['gyro']:
        for d in listOfDataFramesPerClass[k]['gyro'][a]:
            keep_col = ['x','y','z']
            new_df = d[keep_col]
            new_df = new_df[:-24]
            trimmed_df = new_df.iloc[24:]
            if len(trimmed_df) < 50:
                print(str(ctr) + " " + str(len(trimmed_df)))
            trimmedDataFramesPerClass[a].append(trimmed_df)
            ctr += 1
        
    print("Done Trimming " + str(ctr) + " files")
    dataFrameDictionary["gyro"] = trimmedDataFramesPerClass
    listOfTrimmedGyroDataFramesPerClass[k] = dataFrameDictionary

Normalization (For Acceleration and Gyroscope)

In [ ]:
from sklearn import preprocessing

listOfNormalizedAccDataFramesPerClass = {}

for k in listOfTrimmedAccDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = {}
    NormalizedDataFramesPerClass['left'] = []
    NormalizedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['gyro']:
        for d in listOfTrimmedAccDataFramesPerClass[k]['acc'][a]:
            #print(str(len(d.index)) + " " + str(ctr))
            # Create x, where x the 'scores' column's values as floats
            x = d.values.astype(float)

            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()

            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)

            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            #df_normalized
            NormalizedDataFramesPerClass[a].append(df_normalized)
            ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["acc"] = NormalizedDataFramesPerClass
    listOfNormalizedAccDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



In [ ]:
from sklearn import preprocessing

listOfNormalizedGyroDataFramesPerClass = {}

for k in listOfTrimmedGyroDataFramesPerClass.keys():
    print("----------------- Word " + k + "-----------------")
    ctr = 0
    NormalizedDataFramesPerClass = {}
    NormalizedDataFramesPerClass['left'] = []
    NormalizedDataFramesPerClass['right'] = []
    dataFrameDictionary = {}

    for a in listOfDataFramesPerClass[k]['gyro']:
        for d in listOfTrimmedGyroDataFramesPerClass[k]['gyro'][a]:
            #print(str(len(d.index)) + " " + str(ctr))
            # Create x, where x the 'scores' column's values as floats
            x = d.values.astype(float)

            # Create a minimum and maximum processor object
            min_max_scaler = preprocessing.MinMaxScaler()

            # Create an object to transform the data to fit minmax processor
            x_scaled = min_max_scaler.fit_transform(x)

            # Run the normalizer on the dataframe
            df_normalized = pd.DataFrame(x_scaled)
            #df_normalized
            NormalizedDataFramesPerClass[a].append(df_normalized)
            ctr += 1
        
    print("Done Normalizing " + str(ctr) + " files")
    dataFrameDictionary["gyro"] = NormalizedDataFramesPerClass
    listOfNormalizedGyroDataFramesPerClass[k] = dataFrameDictionary  
    
print("Data Normalization done!!")



Label Class

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])

listOfLabeledAccDataPerClass = {}
listOfLabeledImuDataPerClass['right'] = {}
listOfLabeledImuDataPerClass['left'] = {}

for p in listOfNormalizedDataFramesPerClass[k]['emg']:

    for k in listOfTrimmedAccDataFramesPerClass.keys():
        ctr = 0

        labeledDataFramePerClass = []
        labeledDataFrame = pd.DataFrame(columns = ['X of Accelerometer', 'Y of Accelerometer', 'Z of Accelerometer', 
                                                   'X of Gyroscope', 'Y of Gyroscope', 'Z of Gyroscope', 'Class'])
        
        for a, g in zip(listOfNormalizedAccDataFramesPerClass[k]['acc'][p], listOfNormalizedGyroDataFramesPerClass[k]['gyro'][p]):
            xAV = a[0].values
            yAV = a[1].values
            zAV = a[2].values
            xGV = g[2].values
            yGV = g[2].values
            zGV = g[2].values
                        
            
            for x, y, z, xG, yG, zG in zip(xAV, yAV, zAV, xGV, yGV, zGV):
                #print(str(x) + " " + str(y) + " " +str(z))
                labeledDataFrame.loc[ctr] = [x, y, z, xG, yG, zG, k]
                ctr+=1
            print(ctr)

        listOfLabeledImuDataPerClass[p][k] = labeledDataFrame
        print("Done labeling " + p +" imu data of " + k)


In [ ]:
listOfLabeledImuDataPerClass['left']['ano']

In [ ]:
#print(listOfMavDataFramePerClass[0])
#print(listOfRmsDataFramePerClass[0])

listOfLabeledImuDataPerClass = {}
listOfLabeledImuDataPerClass['right'] = {}
listOfLabeledImuDataPerClass['left'] = {}

for p in listOfNormalizedDataFramesPerClass[k]['emg']:

    for k in listOfTrimmedAccDataFramesPerClass.keys():
        ctr = 0

        labeledDataFramePerClass = []
        labeledDataFrame = pd.DataFrame(columns = ['X of Accelerometer', 'Y of Accelerometer', 'Z of Accelerometer', 
                                                   'X of Gyroscope', 'Y of Gyroscope', 'Z of Gyroscope', 'Class'])
        
        for a, g in zip(listOfNormalizedAccDataFramesPerClass[k]['acc'][p],listOfNormalizedGyroDataFramesPerClass[k]['gyro'][p]):
            
            for i in range(0, len(a[0])):
                labeledDataFrame.loc[ctr] = [a[0][i], a[1][i], a[2][i], g[0][i], g[1][i], g[2][i], k]
                ctr+=1
            

        listOfLabeledImuDataPerClass[p][k] = labeledDataFrame
        print("Done labeling " + p +" imu data of " + k)


Data Consolidation (save to csv)

In [ ]:
for a in listOfNormalizedDataFramesPerClass[k]['emg']:
    newDf = listOfLabeledImuDataPerClass[a]['ano']
    for k in listOfTrimmedDataFramesPerClass.keys():
        if k != 'ano':
            print(k)
            newDf= newDf.append(listOfLabeledImuDataPerClass[a][k])
    newDf.to_csv('/Users/user/Desktop/'+ a +'ArmConsolidatedIMU.csv', encoding='utf-8', index=False)

In [ ]:
import glob
import pandas as pd
listOfFileNames = []
listOfClassNames = []
listOfDataPerClass = []
files = []
classNames = []

files = glob.glob("/Users/user/Desktop/datathesis/placeholder/hello/*.csv")
print(len(files))
listOfFileNames = []
for f in files:
    listOfFileNames.append(f.split("\\")[1])
    
emgDataFrames = {}
emgDataFrames['right'] = []
emgDataFrames['left'] = []
    
accDataFrames = {}
accDataFrames['right'] = []
accDataFrames['left'] = []

gyroDataFrames = {}
gyroDataFrames['right'] = []
gyroDataFrames['left'] = []

dataFrameDictionary = {}
leftctr = 0
rightctr = 0
print(len(listOfFileNames))
for fileName in listOfFileNames:
    fileLocation = "/Users/user/Desktop/datathesis/placeholder/hello/"+fileName 
    #print(fileLocation)
    df = pd.read_csv(fileLocation)
    
    if "emg" in fileName:   
        if "-0-" in fileName:
            emgDataFrames['right'].append(df)
            leftctr+=1
        else:
            emgDataFrames['left'].append(df)
            rightctr+=1
    elif ("accelerometer") in fileName:
        if "-0-" in fileName:
            accDataFrames['right'].append(df)
            leftctr+=1
        else:
            accDataFrames['left'].append(df)
            rightctr+=1
    elif ("gyro") in fileName:
        if "-0-" in fileName:
            gyroDataFrames['right'].append(df)
            leftctr+=1
        else:
            gyroDataFrames['left'].append(df)      
            rightctr+=1

print("Total of " + str(leftctr) + " left arm files and " + str(rightctr) + " right arm files.")

    
#listOfClassNames
#listOfDataPerClass[0]
print("Done listing the class and file names")